# Importações

In [2]:
import os
import pydicom
from pydicom.misc import is_dicom
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk

In [10]:
def print_image(image):

    # Converte a imagem para um numpy array
    image_array = image.numpy()

    # Obtém o índice central para cada dimensão
    axial_index = image_array.shape[0] // 2  # Corte axial
    coronal_index = image_array.shape[1] // 2  # Corte coronal
    sagital_index = image_array.shape[2] // 2  # Corte sagital

    # Cria um gráfico com 3 subgráficos
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Exibe o corte axial
    axes[0].imshow(image_array[:, :, axial_index], cmap='gray')
    axes[0].set_title('Corte Axial')
    axes[0].axis('off')

    # Exibe o corte coronal
    axes[1].imshow(image_array[:, coronal_index, :], cmap='gray')
    axes[1].set_title('Corte Coronal')
    axes[1].axis('off')

    # Exibe o corte sagital
    axes[2].imshow(image_array[sagital_index, :, :], cmap='gray')
    axes[2].set_title('Corte Sagital')
    axes[2].axis('off')

    # Ajusta o layout e exibe os cortes
    plt.tight_layout()
    plt.show()

In [ ]:
import os

DIR_BASE = os.path.abspath('/mnt/d/ADNI/ADNI1')
DIR_RAW = os.path.join(DIR_BASE, 'ADNI_nii_raw')
DIR_OUTPUT = os.path.join(DIR_BASE, 'ADNI_nii_processed')

raw = [(item) for item in os.listdir(DIR_RAW)]
processed = [(item) for item in os.listdir(DIR_OUTPUT)]

for item in processed:
    if item in raw:
        raw.remove(item)

for item in raw:
    print(item)

In [3]:
import pandas as pd

# Carregar o arquivo CSV ADNI1
adni1_csv = '/mnt/c/Users/bruno/Desktop/IANS/alzheimer_project/codes/mri/ADNI1_Screening_10_04_2024.csv' 
adni1_df = pd.read_csv(adni1_csv)

# Carregar o arquivo CSV ADNI2
adni2_csv = '/mnt/c/Users/bruno/Desktop/IANS/alzheimer_project/codes/mri/ADNI_2_Screening_-_New_Pt_10_23_2024.csv'
adni2_df = pd.read_csv(adni2_csv)

adni1 = adni1_df['Subject'].tolist()
adni2 = adni2_df['Subject'].tolist()

for item in adni2:
    if item in adni1:
        print(item)

In [ ]:
import os
import pandas as pd

# Diretorio
directory_adni1 = os.listdir('/mnt/d/ADNI/ADNI1/ADNI_nii_processed')

# Converte a coluna 'Subject' em uma lista
subjects1 = {'cn': [], 'mci': [], 'ad': []}

for item in directory_adni1:
    name = item.rsplit('.', 2)[0]
    line = adni1_df[adni1_df['Subject'] == name]

    group = line['Group'].iloc[0]
    
    if group == 'CN':
        subjects1['cn'].append(name + '.nii.gz')
    elif group == 'MCI':
        subjects1['mci'].append(name + '.nii.gz')
    elif group == 'AD':
        subjects1['ad'].append(name + '.nii.gz')

# Conta as ocorrências dos arquivos na lista dir
print(subjects1['cn'])
print(subjects1['mci'])
print(subjects1['ad'])

print(len(subjects1['cn']))
print(len(subjects1['mci']))
print(len(subjects1['ad']))



In [ ]:
import pandas as pd

directory_adni2 = os.listdir('/mnt/d/ADNI/ADNI2/ADNI_nii_processed')

# Converte a coluna 'Subject' em uma lista
subjects2 = {'cn': [], 'mci': [], 'ad': []}

for item in adni2:
    line = adni2_df[adni2_df['Subject'] == item]

    group = line['Group'].iloc[0]
    
    if group == 'CN':
        subjects2['cn'].append(item + '.nii.gz')
    elif group == 'MCI':
        subjects2['mci'].append(item + '.nii.gz')
    elif group == 'AD':
        subjects2['ad'].append(item + '.nii.gz')

# Conta as ocorrências dos arquivos na lista dir
print(subjects2['cn'])
print(subjects2['mci'])
print(subjects2['ad'])

print(len(subjects2['cn']))
print(len(subjects2['mci']))
print(len(subjects2['ad']))

In [ ]:
import pandas as pd

# Converte a coluna 'Subject' em uma lista
subjects_total = {'cn': [], 'mci': [], 'ad': []}

for item in directory_adni1:
    name = item.rsplit('.', 2)[0]
    line = adni1_df[adni1_df['Subject'] == name]

    group = line['Group'].iloc[0]
    
    if group == 'CN':
        subjects_total['cn'].append(name + '.nii.gz')
    elif group == 'MCI':
        subjects_total['mci'].append(name + '.nii.gz')
    elif group == 'AD':
        subjects_total['ad'].append(name + '.nii.gz')

for item in directory_adni2:
    name = item.rsplit('.', 2)[0]
    line = adni2_df[adni2_df['Subject'] == name]

    group = line['Group'].iloc[0]
    
    if group == 'CN':
        subjects_total['cn'].append(name +'.nii.gz')
    elif group == 'MCI':
        subjects_total['mci'].append(name + '.nii.gz')
    elif group == 'AD':
        subjects_total['ad'].append(name + '.nii.gz')


# Conta as ocorrências dos arquivos na lista dir
print(subjects_total['cn'])
print(subjects_total['mci'])
print(subjects_total['ad'])

print(len(subjects_total['cn']))
print(len(subjects_total['mci']))
print(len(subjects_total['ad']))

In [ ]:
import random
import math

# Dividir pastas com arredondamento
def precise_distributor(subjects, new_dir, percentages):
    # Inicializando o número total de sujeitos em cada categoria
    total_cn = len(subjects['cn'])
    total_mci = len(subjects['mci'])
    total_ad = len(subjects['ad'])

    # Calculando a quantidade exata para cada conjunto, para cada categoria
    train_cn = math.floor(percentages[0] * total_cn)
    validation_cn = math.floor(percentages[1] * total_cn)
    test_cn = total_cn - train_cn - validation_cn

    train_mci = math.floor(percentages[0] * total_mci)
    validation_mci = math.floor(percentages[1] * total_mci)
    test_mci = total_mci - train_mci - validation_mci

    train_ad = math.floor(percentages[0] * total_ad)
    validation_ad = math.floor(percentages[1] * total_ad)
    test_ad = total_ad - train_ad - validation_ad

    # Distribuindo CN
    for _ in range(train_cn):
        id = random.choice(subjects['cn'])
        new_dir['train']['cn'].append(id)
        subjects['cn'].remove(id)

    for _ in range(validation_cn):
        id = random.choice(subjects['cn'])
        new_dir['validation']['cn'].append(id)
        subjects['cn'].remove(id)

    for _ in range(test_cn):
        id = random.choice(subjects['cn'])
        new_dir['test']['cn'].append(id)
        subjects['cn'].remove(id)

    # Distribuindo MCI
    for _ in range(train_mci):
        id = random.choice(subjects['mci'])
        new_dir['train']['mci'].append(id)
        subjects['mci'].remove(id)

    for _ in range(validation_mci):
        id = random.choice(subjects['mci'])
        new_dir['validation']['mci'].append(id)
        subjects['mci'].remove(id)

    for _ in range(test_mci):
        id = random.choice(subjects['mci'])
        new_dir['test']['mci'].append(id)
        subjects['mci'].remove(id)

    # Distribuindo AD
    for _ in range(train_ad):
        id = random.choice(subjects['ad'])
        new_dir['train']['ad'].append(id)
        subjects['ad'].remove(id)

    for _ in range(validation_ad):
        id = random.choice(subjects['ad'])
        new_dir['validation']['ad'].append(id)
        subjects['ad'].remove(id)

    for _ in range(test_ad):
        id = random.choice(subjects['ad'])
        new_dir['test']['ad'].append(id)
        subjects['ad'].remove(id)

# Inicializando pastas de treino, validação e teste
data_split = {
    'train': {'cn': [], 'mci': [], 'ad': []},
    'validation': {'cn': [], 'mci': [], 'ad': []},
    'test': {'cn': [], 'mci': [], 'ad': []}
}

# Proporções desejadas: 70% treino, 20% validação, 10% teste
proportions = [0.7, 0.2, 0.1]

# Distribuir os sujeitos
precise_distributor(subjects_total, data_split, proportions)

# Verificando resultados
print("Treino CN:", len(data_split['train']['cn']), "MCI:", len(data_split['train']['mci']), "AD:", len(data_split['train']['ad']))
print("Validação CN:", len(data_split['validation']['cn']), "MCI:", len(data_split['validation']['mci']), "AD:", len(data_split['validation']['ad']))
print("Teste CN:", len(data_split['test']['cn']), "MCI:", len(data_split['test']['mci']), "AD:", len(data_split['test']['ad']))


In [ ]:
#INFORMAÇÕES

train = data_split['train']
validation = data_split['validation']
test = data_split['test']

print("FULL DATASET:")
print(f"CN: {len(subjects_total['cn'])}   MCI: {len(subjects_total['mci'])}   AD: {len(subjects_total['ad'])}")
print(f"{subjects_total['cn']}\n{subjects_total['mci']}\n{subjects_total['ad']}\n\n")

print("TRAIN DATASET:")
print(f"CN: {len(train['cn'])}   MCI: {len(train['mci'])}   AD: {len(train['ad'])}")
print(f"{train['cn']}\n{train['mci']}\n{train['ad']}\n\n")

print("VALIDATION DATASET:")
print(f"CN: {len(validation['cn'])}   MCI: {len(validation['mci'])}   AD: {len(validation['ad'])}")
print(f"{validation['cn']}\n{validation['mci']}\n{validation['ad']}\n\n")

print("TEST DATASET")
print(f"CN: {len(test['cn'])}   MCI: {len(test['mci'])}   AD: {len(test['ad'])}")
print(f"{test['cn']}\n{test['mci']}\n{test['ad']}\n\n")

In [ ]:
import shutil

# DIRETÓRIOS ORIGINAIS
dir_adni1 = '/mnt/d/ADNI/ADNI1/ADNI_nii_processed'
dir_adni2 = '/mnt/d/ADNI/ADNI2/ADNI_nii_processed'

# DIRETÓRIO FINAL
dir_final = '/mnt/d/ADNI/ADNI1_ADNI2'

for subset in data_split: # pega chaves train, validation e test do dicionário inicial
    dir_copy = os.path.join(dir_final, subset)
    for category in data_split[subset]: # pega chaves cn, mci e ad
        for item in data_split[subset][category]: # pega arquivos
            if item in os.listdir(dir_adni1):
                file = os.path.join(dir_adni1, item)
            elif item in os.listdir(dir_adni2):
                file = os.path.join(dir_adni2, item)
            shutil.copy(file, os.path.join(dir_copy, category, item))
        print(f"ARQUIVOS DE {subset} DA CATEGORIA {category} COPIADOS")



In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Caminho para a imagem NIfTI
img_path = "/mnt/c/Users/bruno/Desktop/IANS/ADNI1_ADNI2/train/ad/002_S_0619.nii.gz"

# Carregar a imagem NIfTI
img_nifti = nib.load(img_path)
img_array = img_nifti.get_fdata()  # Extrai os dados da imagem

# Normalização (opcional, dependendo de como você processa seus dados)
img_array = img_array / np.max(img_array)  # Normaliza para [0, 1]

# Calcular a média e o desvio padrão
mean = np.mean(img_array)
std_dev = np.std(img_array)

print(f'Média dos pixels: {mean:.4f}')
print(f'Desvio padrão dos pixels: {std_dev:.4f}')

# Visualizar a imagem
# Como as imagens NIfTI são 3D, você pode precisar escolher um corte
slice_index = img_array.shape[2] // 2  # Pegue o meio do volume
plt.imshow(img_array[:, :, slice_index], cmap='gray')
plt.title('Imagem Original (Corte)')
plt.axis('off')
plt.show()